In [1]:
def clean_text_advanced(text):
    """
    Limpia y normaliza el texto eliminando frases específicas de formato
    """
    if not text or pd.isna(text):
        return ""

    # Convertir a string
    text = str(text).strip()

    # Lista de frases/patrones a eliminar (específicos de tus documentos)
    phrases_to_remove = [
        r'Fundamental el nombre preciso',
        r'Slogan',
        r'hasta \d+ renglones? aprox\.?',
        r'hasta \d+ renglones?',
        r'aprox\.?',
        r'máximo \d+ renglones?',
        r'mínimo \d+ renglones?',
        r'\d+ renglones? como máximo',
        r'\d+ renglones? como mínimo',
        r'completar con.*?información',
        r'llenar con.*?datos',
        r'incluir.*?información.*?relevante',
        r'agregar.*?detalles',
        r'especificar.*?datos',
        r'indicar.*?información',
        r'colocar.*?datos',
        r'escribir.*?información',
        r'formato.*?requerido',
        r'campo.*?obligatorio',
        r'sección.*?completar',
        r'ejemplo:.*?$',
        r'ej:.*?$',
        r'nota:.*?$',
        r'observación:.*?$',
        r'aclaración:.*?$'
    ]

    # Eliminar frases específicas
    for pattern in phrases_to_remove:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.MULTILINE)

    # Eliminar patrones comunes de formato
    format_patterns = [
        r'\*+',  # Asteriscos múltiples
        r'_{2,}',  # Guiones bajos múltiples
        r'-{2,}',  # Guiones múltiples
        r'={2,}',  # Signos igual múltiples
        r'\s*\([^)]*renglón[^)]*\)',  # Texto entre paréntesis que mencione "renglón"
        r'\s*\[[^\]]*renglón[^\]]*\]',  # Texto entre corchetes que mencione "renglón"
        r'^\s*[\d\.]+\s*$',  # Líneas que solo contengan números
        r'^\s*[IVX]+\.\s*$',  # Numeración romana
        r'^\s*[a-z]\)\s*$',  # Letras con paréntesis (a), b), etc.)
        r'\s*\.\.\.\s*',  # Puntos suspensivos
        r'(?:campo|sección|apartado)\s*\d+',  # Referencias a campos numerados
        r'ver\s+(?:anexo|ejemplo|modelo)',  # Referencias a anexos/ejemplos
    ]

    for pattern in format_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.MULTILINE)

    # Limpiezas generales
    text = re.sub(r'\s+', ' ', text)  # Múltiples espacios a uno solo
    text = re.sub(r'\n+', ' ', text)  # Múltiples saltos de línea a espacio
    text = re.sub(r'[^\w\s\-.,;:()áéíóúüñÁÉÍÓÚÜÑ¿¡]', '', text)  # Caracteres especiales

    # Eliminar espacios al inicio y final
    text = text.strip()

    # Si después de limpiar queda muy poco texto, devolver vacío
    if len(text) < 3:
        return ""

    return text

def add_custom_cleanup_phrases(additional_phrases):
    """
    Función para agregar más frases específicas que encuentres en tus documentos
    """
    global custom_cleanup_phrases
    custom_cleanup_phrases = additional_phrases

    print(f"✅ Agregadas {len(additional_phrases)} frases adicionales para limpieza:")
    for phrase in additional_phrases:
        print(f"   - {phrase}")

def clean_dataframe(df):
    """
    Aplica la limpieza avanzada a todo el DataFrame
    """
    print("🧹 Aplicando limpieza avanzada...")

    # Campos de texto para limpiar (excluir campos técnicos)
    text_fields = [col for col in df.columns
                   if col not in ['archivo_fuente']]

    # Aplicar limpieza a cada campo
    for field in text_fields:
        if field in df.columns:
            print(f"   Limpiando campo: {field}")
            df[field] = df[field].apply(clean_text_advanced)

    print("✅ Limpieza completada")
    return df

def compare_before_after(df_original, df_cleaned, field='mision', sample_size=3):
    """
    Compara los datos antes y después de la limpieza
    """
    print(f"🔍 COMPARACIÓN ANTES/DESPUÉS - Campo: {field}")
    print("=" * 80)

    for i in range(min(sample_size, len(df_original))):
        if pd.notna(df_original.iloc[i][field]) and df_original.iloc[i][field] != '':
            print(f"\n📄 EJEMPLO {i+1}:")
            print(f"ANTES: {df_original.iloc[i][field][:200]}...")
            print(f"DESPUÉS: {df_cleaned.iloc[i][field][:200]}...")
            print("-" * 40)

# FUNCIÓN PARA REEMPLAZAR EN TU SCRIPT ORIGINAL
def parse_ong_document_improved(text):
    """
    Versión mejorada de parse_ong_document con limpieza avanzada
    """
    # Primero aplicar limpieza avanzada al texto completo
    cleaned_text = clean_text_advanced(text)

    ong_data = {
        'nombre': '',
        'mensaje_descriptivo': '',
        'presentacion': '',
        'quienes_son': '',
        'mision': '',
        'vision': '',
        'servicios': '',
        'poblacion_beneficiaria': '',
        'que_ofrece': '',
        'periodicidad': '',
        'horarios': '',
        'condiciones_acceso': '',
        'categoria': '',
        'ubicacion': '',
        'contacto': '',
        'encargado': '',
        'texto_completo': cleaned_text
    }

    # Patrones para identificar secciones (igual que antes)
    patterns = {
        'nombre': [r'nombre.*?:?\s*(.+?)(?:\n|$)', r'^(.+?)(?:\n|mensaje|presentaci)', r'ONG\s*:?\s*(.+?)(?:\n|$)'],
        'mensaje_descriptivo': [r'mensaje\s*descriptivo.*?:?\s*(.+?)(?:\n\n|\npresentaci)', r'descripci[oó]n.*?:?\s*(.+?)(?:\n\n|\nquienes)'],
        'presentacion': [r'presentaci[oó]n.*?:?\s*(.+?)(?:\n\n|\nquienes|\nmisi)', r'presentaci[oó]n\s*de\s*la\s*ong.*?:?\s*(.+?)(?:\n\n|\nquienes)'],
        'quienes_son': [r'qui[eé]nes?\s*son.*?:?\s*(.+?)(?:\n\n|\nmisi|\nvisi)', r'quienes\s*somos.*?:?\s*(.+?)(?:\n\n|\nmisi)'],
        'mision': [r'misi[oó]n.*?:?\s*(.+?)(?:\n\n|\nvisi|\nservici)', r'nuestra\s*misi[oó]n.*?:?\s*(.+?)(?:\n\n|\nvisi)'],
        'vision': [r'visi[oó]n.*?:?\s*(.+?)(?:\n\n|\nservici|\npoblaci)', r'nuestra\s*visi[oó]n.*?:?\s*(.+?)(?:\n\n|\nservici)'],
        'servicios': [r'servici.*?:?\s*(.+?)(?:\n\n|\npoblaci|\nque\s*ofrece)', r'descripci[oó]n.*?servici.*?:?\s*(.+?)(?:\n\n|\npoblaci)'],
        'poblacion_beneficiaria': [r'poblaci[oó]n\s*beneficiari.*?:?\s*(.+?)(?:\n\n|\nque\s*ofrece|\nperiodicidad)', r'beneficiari.*?:?\s*(.+?)(?:\n\n|\nque\s*ofrece)'],
        'que_ofrece': [r'qu[eé]\s*ofrece.*?:?\s*(.+?)(?:\n\n|\nperiodicidad|\nhorari)', r'ofrecemos.*?:?\s*(.+?)(?:\n\n|\nperiodicidad)'],
        'periodicidad': [r'periodicidad.*?:?\s*(.+?)(?:\n\n|\nhorari|\ncondici)', r'frecuencia.*?:?\s*(.+?)(?:\n\n|\nhorari)'],
        'horarios': [r'horari.*?:?\s*(.+?)(?:\n\n|\ncondici|\ncategor)', r'horarios?\s*de\s*atenci[oó]n.*?:?\s*(.+?)(?:\n\n|\ncondici)'],
        'condiciones_acceso': [r'condici.*?acceso.*?:?\s*(.+?)(?:\n\n|\ncategor|\nubicaci)', r'requisitos.*?:?\s*(.+?)(?:\n\n|\ncategor)'],
        'categoria': [r'categor[ií]a.*?:?\s*(.+?)(?:\n\n|\nubicaci|\ncontact)', r'tipo.*?:?\s*(.+?)(?:\n\n|\nubicaci)'],
        'ubicacion': [r'ubicaci[oó]n.*?:?\s*(.+?)(?:\n\n|\ncontact|\nencargad)', r'direcci[oó]n.*?:?\s*(.+?)(?:\n\n|\ncontact)'],
        'contacto': [r'contact.*?:?\s*(.+?)(?:\n\n|\nencargad|$)', r'tel[eé]fono.*?:?\s*(.+?)(?:\n\n|\nencargad)'],
        'encargado': [r'encargad.*?:?\s*(.+?)(?:\n\n|$)', r'responsable.*?:?\s*(.+?)(?:\n\n|$)']
    }

    # Normalizar texto para búsqueda
    text_lower = cleaned_text.lower()

    # Extraer información usando patrones
    for field, field_patterns in patterns.items():
        for pattern in field_patterns:
            match = re.search(pattern, text_lower, re.DOTALL | re.IGNORECASE)
            if match:
                extracted = match.group(1).strip()
                if extracted and len(extracted) > 2:
                    # Aplicar limpieza adicional al texto extraído
                    clean_extracted = clean_text_advanced(extracted)
                    if clean_extracted:  # Solo si queda algo después de limpiar
                        ong_data[field] = clean_extracted[:500]  # Limitar longitud
                        break

    return ong_data

# INSTRUCCIONES DE USO
print("""
🧹 LIMPIEZA AVANZADA DE DATOS

Para usar estas funciones mejoradas:

1. Si ya tienes un DataFrame procesado:
   df_cleaned = clean_dataframe(df_original.copy())

2. Para comparar antes/después:
   compare_before_after(df_original, df_cleaned, field='mision')

3. Para agregar más frases específicas:
   frases_adicionales = ['otra frase a eliminar', 'más texto innecesario']
   add_custom_cleanup_phrases(frases_adicionales)

4. Para reprocesar desde cero con limpieza mejorada:
   Reemplaza parse_ong_document() por parse_ong_document_improved()
   en tu script original

¡Esto debería eliminar todas esas frases de formato! 🎉
""")


🧹 LIMPIEZA AVANZADA DE DATOS

Para usar estas funciones mejoradas:

1. Si ya tienes un DataFrame procesado:
   df_cleaned = clean_dataframe(df_original.copy())

2. Para comparar antes/después:
   compare_before_after(df_original, df_cleaned, field='mision')

3. Para agregar más frases específicas:
   frases_adicionales = ['otra frase a eliminar', 'más texto innecesario']
   add_custom_cleanup_phrases(frases_adicionales)

4. Para reprocesar desde cero con limpieza mejorada:
   Reemplaza parse_ong_document() por parse_ong_document_improved()
   en tu script original

¡Esto debería eliminar todas esas frases de formato! 🎉

